In [1]:
import pandas as pd
import requests
from pandas_profiling import ProfileReport
import sweetviz as sv
import numpy as np

In [62]:
unnecessary_columns = ['V_ID','P_ID','C_YEAR','P_USER','P_PSN','V_YEAR','P_ISEV']

In [63]:
data = pd.read_csv('NCDB_1999_to_2014.csv',low_memory=False)

In [64]:
data

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
2,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,02,F,20,13,2,02,2
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
4,1999,01,1,08,2,01,01,UU,5,3,...,99,NN,NNNN,01,M,05,99,2,UU,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860400,2014,UU,U,UU,2,UU,UU,01,U,U,...,13,07,UUUU,01,M,24,11,1,NN,1
5860401,2014,UU,U,23,2,01,06,05,1,1,...,01,14,2006,01,M,29,96,2,09,5
5860402,2014,UU,U,14,2,01,02,01,1,5,...,01,01,2006,01,F,UU,11,2,01,1
5860403,2014,UU,U,22,1,01,06,01,2,4,...,01,22,UUUU,01,M,67,12,3,01,U


In [65]:
#Agregar columna con cantidad de personas por accidente
collision_columns = [col for col in data.columns if col[0]=="C"]
#group_by_columns = collision_columns[:]
#group_by_columns.append('P_ID')
#collision_people_count_df = data[group_by_columns].groupby(group_by_columns).size().reset_index().drop('P_ID',axis=1).rename(columns={0:"A_PERS"}).drop_duplicates()
collision_people_count_df = data.groupby(collision_columns).size().reset_index().rename(columns={0:"A_PERS"}).drop_duplicates()

In [66]:
collision_people_count_df

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,A_PERS
0,1999,01,1,00,2,01,01,01,1,4,1,18,1
1,1999,01,1,00,2,01,01,01,1,5,1,18,1
2,1999,01,1,00,2,01,01,01,2,1,1,18,1
3,1999,01,1,00,2,01,01,01,6,3,2,18,2
4,1999,01,1,00,2,01,01,UU,1,2,3,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850390,2014,12,7,UU,2,02,UU,02,U,U,1,03,2
1850391,2014,UU,U,14,2,01,02,01,1,5,4,18,1
1850392,2014,UU,U,22,1,01,06,01,2,4,Q,18,2
1850393,2014,UU,U,23,2,01,06,05,1,1,U,18,1


In [67]:
#quedarse solo con los drivers
data = data[data['P_PSN']=="11"]
data

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
5,1999,01,1,17,2,03,QQ,QQ,1,2,...,01,01,1984,01,M,28,11,1,UU,1
6,1999,01,1,17,2,03,QQ,QQ,1,2,...,02,01,1991,01,M,21,11,1,UU,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860397,2014,UU,U,UU,2,UU,UU,01,U,U,...,10,01,UUUU,01,F,79,11,1,NN,1
5860398,2014,UU,U,UU,2,UU,UU,01,U,U,...,11,01,UUUU,01,F,20,11,1,NN,1
5860399,2014,UU,U,UU,2,UU,UU,01,U,U,...,12,01,UUUU,01,F,47,11,1,NN,1
5860400,2014,UU,U,UU,2,UU,UU,01,U,U,...,13,07,UUUU,01,M,24,11,1,NN,1


In [68]:
data = data.merge(collision_people_count_df,on=collision_columns,how='left')

In [69]:
print(f'Numero de registros: {data.shape[0]}')
print(f'Numero de columnas: {data.shape[1]}')

Numero de registros: 3926086
Numero de columnas: 23


In [70]:
data.dtypes.to_dict()

{'C_YEAR': dtype('int64'),
 'C_MNTH': dtype('O'),
 'C_WDAY': dtype('O'),
 'C_HOUR': dtype('O'),
 'C_SEV': dtype('int64'),
 'C_VEHS': dtype('O'),
 'C_CONF': dtype('O'),
 'C_RCFG': dtype('O'),
 'C_WTHR': dtype('O'),
 'C_RSUR': dtype('O'),
 'C_RALN': dtype('O'),
 'C_TRAF': dtype('O'),
 'V_ID': dtype('O'),
 'V_TYPE': dtype('O'),
 'V_YEAR': dtype('O'),
 'P_ID': dtype('O'),
 'P_SEX': dtype('O'),
 'P_AGE': dtype('O'),
 'P_PSN': dtype('O'),
 'P_ISEV': dtype('O'),
 'P_SAFE': dtype('O'),
 'P_USER': dtype('O'),
 'A_PERS': dtype('int64')}

In [71]:
nas = ["UU","XX","U","X","XXXX","NN","NNNN","N","UUUU"]
others = {
    "P_SAFE" : "14",
    "P_PSN": "100",
    "V_TYPE": "24",
    "C_TRAF": "19",
    "C_RALN": "7",
    "C_RSUR": "10",
    "C_WTHR": "8",
    "C_RCFG": "13",
    "C_CONF": "42"
}
int_types = ["P_AGE","A_VAGE","A_PERS","C_VEHS"]
categories = [col for col in data.columns if col not in int_types + unnecessary_columns]
target_columns = ["C_SEV","P_ISEV"]

In [72]:
print("|".join(nas))

UU|XX|U|X|XXXX|NN|NNNN|N|UUUU


In [73]:
#data['C_MNTH'].replace(to_replace=rf'/^({"|".join(nas)})$/',value=pd.NA,regex=True)
#data['C_MNTH'].replace(to_replace=rf'^({"|".join(nas)})',value=pd.NA,regex=True)

In [74]:
#for na in nas:
#    data[categories] = data[categories].replace(na,pd.NA)

data[categories] = data[categories].replace(to_replace=rf'^({"|".join(nas)})',value=pd.NA,regex=True)

for key,value in others.items():
    data[key] = data[key].replace(to_replace=r'^Q.*', value=value, regex=True)
    
#data.replace(to_replace=r'^[A-Z].*',value=np.nan, regex=True,inplace=True)

In [75]:
categories

['C_MNTH',
 'C_WDAY',
 'C_HOUR',
 'C_SEV',
 'C_CONF',
 'C_RCFG',
 'C_WTHR',
 'C_RSUR',
 'C_RALN',
 'C_TRAF',
 'V_TYPE',
 'P_SEX',
 'P_SAFE']

In [76]:
data['C_MNTH']

0            01
1            01
2            01
3            01
4            01
           ... 
3926081    <NA>
3926082    <NA>
3926083    <NA>
3926084    <NA>
3926085    <NA>
Name: C_MNTH, Length: 3926086, dtype: object

In [77]:
#agregar columna con antiguedad del coche
data['A_VAGE'] = data['C_YEAR'] - pd.to_numeric(data['V_YEAR'], errors='coerce')

In [78]:
#convertir las horas en 4 categorias
def transform_hour(value):
    value = pd.to_numeric(value,errors='coerce')
    if value >= 0 and value<6:
        return '0'
    if value >= 6 and value<12:
        return '1'
    if value >= 12 and value<18:
        return '2'
    if value >= 18 and value<24:
        return '3'
    return pd.NA

data['C_HOUR'] = data['C_HOUR'].apply(transform_hour) 
    

In [79]:
data[int_types] = data[int_types].apply(pd.to_numeric, errors='coerce',downcast='integer')
data[categories] = data[categories].astype('category')

In [80]:
data = data.rename(columns={'P_SEX':'A_DSEX','P_AGE':"A_DAGE",'C_HOUR':'A_CHUR'})

In [81]:
data = data.drop(unnecessary_columns,1).drop_duplicates()
data

,C_MNTH,C_WDAY,A_CHUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_TYPE,A_DSEX,A_DAGE,P_SAFE,A_PERS,A_VAGE
0,01,1,3,2,2.0,34,NaN,1,5,3,03,06,M,41.0,NaN,3,9.0
1,01,1,3,2,2.0,34,NaN,1,5,3,03,01,M,19.0,NaN,3,12.0
2,01,1,1,2,1.0,01,NaN,5,3,6,18,01,M,46.0,NaN,2,13.0
3,01,1,2,2,3.0,42,13,1,2,1,01,01,M,28.0,NaN,4,15.0
4,01,1,2,2,3.0,42,13,1,2,1,01,01,M,21.0,NaN,4,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926070,12,7,0,2,1.0,04,01,2,2,1,18,01,M,49.0,02,1,20.0
3926071,12,7,3,2,1.0,02,01,2,5,1,18,01,M,20.0,02,2,4.0
3926072,12,7,2,2,2.0,41,01,1,5,1,18,05,M,44.0,02,3,6.0
3926073,12,7,3,2,1.0,03,01,1,3,4,18,01,F,35.0,02,3,13.0


In [82]:
data["C_MNTH"]

0           01
1           01
2           01
3           01
4           01
          ... 
3926070     12
3926071     12
3926072     12
3926073     12
3926085    NaN
Name: C_MNTH, Length: 3889560, dtype: category
Categories (12, object): ['01', '02', '03', '04', ..., '09', '10', '11', '12']

In [83]:
data.dtypes.to_dict()

{'C_MNTH': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12'],
                  ordered=False),
 'C_WDAY': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7'], ordered=False),
 'A_CHUR': CategoricalDtype(categories=['0', '1', '2', '3'], ordered=False),
 'C_SEV': CategoricalDtype(categories=[1, 2], ordered=False),
 'C_VEHS': dtype('float64'),
 'C_CONF': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '21', '22', '23', '24',
                   '25', '31', '32', '33', '34', '35', '36', '41', '42'],
                  ordered=False),
 'C_RCFG': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '13'],
                  ordered=False),
 'C_WTHR': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7', '8'], ordered=False),
 'C_RSUR': CategoricalDtype(categories=['1', '10', '2', '3', '4', '5', '6', '7', '8', '9'], ordered=False),
 

In [84]:
data.shape

(3889560, 17)

In [86]:
data.to_csv('crash_transformed.csv',index=False)

In [ ]:
#my_report = sv.analyze(data)
#my_report.show_notebook()
profile = ProfileReport(data, title="Crashes EDA")
profile.to_widgets()